In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [8]:
import warnings
import os
import logging
# 경고 제거
warnings.filterwarnings('ignore')

# transformers 로깅 레벨 조정
logging.getLogger("transformers").setLevel(logging.ERROR)

# Hugging Face symlink 경고 제거
os.environ['HF_HUB_DISABLE_SYMLINKS_WARNING'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# from transformers import pipeline, logging as hf_logging
# hf_logging.set_verbosity_error()

# <span style="color:red">ch1. 허깅페이스</span>
- Inference API 이용 : 모델의 결과를 surver에서 
- pipeline() 이용 : 모델을 다운로드받아 모델의 결과를 local에서
    * raw text -> tokenizer -> model -> [0.11, 0.55, 0.xx, ~] logits값으로 prediction 결과 출력
```
허깅페이스 transformers에서 지원하는 task
"sentiment-analysis" : "text-classification"의 별칭(감정분석 전용으로 사용)
"text-classification" : 감정분석, 뉴스분류, 리뷰 분류 등 일반적인 문장 분류
"zero-shot-classification" : 레이블을 학습 없이 주어진 후보군 중에서 분류
"token-classification" : 개체명 인식(NER ; Named Entity REcognition) 등 단위 라벨링
"ner" : "token-classification"의 별칭
"fill-mask" : 빈칸 채우기
"text-generation" : 텍스트 생성 (GPT류 모델에 사용)
"text2text-generation" : 번역, 요약 등 입력 -> 출력 변환 
"translation" : 번역
"summarization" : 텍스트요약
"question-answering" : 주어진 context를 보고 질문에 답하기.
"image-to-text" : 그림을 설명
"image-classification" : 이미지분류
```

## 1. 텍스트 기반 감정분석(긍정/부정)
- c:/사용자/내컴퓨터명/.cache/huggingface/hub 모델 다운로드

In [6]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [2]:
from transformers import pipeline
classifier = pipeline(task="text-classification",
                     model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
# 감정분석시 내용이 많으면 list로
classifier([
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!"
])

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [4]:
classifier(["이 영화 정말 최고였어요. 감동적이고 연기가 대단해",
            "This movie was the best. It's touching, and the acting is amazing"])

[{'label': 'POSITIVE', 'score': 0.8578149676322937},
 {'label': 'POSITIVE', 'score': 0.9998821020126343}]

In [5]:
classifier("이 물건 정말 사고 싶어요")

[{'label': 'POSITIVE', 'score': 0.8577594757080078}]

In [6]:
classifier(["I like you", "I hat you", "나 너가 싫어", "힘들어요"])

[{'label': 'POSITIVE', 'score': 0.9998695850372314},
 {'label': 'POSITIVE', 'score': 0.999488353729248},
 {'label': 'NEGATIVE', 'score': 0.5993236303329468},
 {'label': 'POSITIVE', 'score': 0.8669536709785461}]

In [6]:
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis",
                     model="matthewburke/korean_sentiment")
texts = ['나는 너가 좋아', "당신이 싫어요", "힘들어요", "오늘 기분이 최고야"]
result = classifier(texts)

Device set to use cpu


In [10]:
for text, result in zip(texts, classifier(texts)):
    label = "긍정" if result['label']=='LABEL_1' else "부정"
    print(f"{text} => {label} : {result['score']:.4f}")

나는 너가 좋아 => 긍정 : 0.9558
당신이 싫어요 => 부정 : 0.9093
힘들어요 => 부정 : 0.9140
오늘 기분이 최고야 => 긍정 : 0.9714


## 2. 제로샷분류(Zero-shot분류)
- 기계학습 및 자연어처리에서 각 개별 작업에 대한 특정 교육없이 작업을 수행할 수 있는 모형(비지도학습)

In [13]:
classifier = pipeline("zero-shot-classification",
                     # model="facebook/bart-large-mnli"
                     )
classifier(
    "I have a problem with my iphone that needs to be resolved asap!",
    candidate_labels=["urgent", "not urgent", "phone", "tablet", "computer"]
)

Device set to use cpu


{'sequence': 'I have a problem with my iphone that needs to be resolved asap!',
 'labels': ['urgent', 'phone', 'computer', 'not urgent', 'tablet'],
 'scores': [0.5227578282356262,
  0.4581405818462372,
  0.014264547266066074,
  0.0026850118301808834,
  0.0021520592272281647]}

In [14]:
sequence_to_classify = "One day I well see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)

{'sequence': 'One day I well see the world',
 'labels': ['travel', 'cooking', 'dancing'],
 'scores': [0.9938077330589294, 0.0030999132432043552, 0.003092374885454774]}

## 3. text 생성

In [20]:
from transformers import pipeline, set_seed
# set_seed(2)
generation = pipeline("text-generation", "gpt2") # 텍스트 생성 gpt3부터는 허깅페이스없음
generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) # pad_token_id 경고를 없애려고 setting

Device set to use cpu


[{'generated_text': 'in this course. We will teach you how to build your own tools to solve your own problems. We will teach you how to create code that works for you. We will teach you how to use a project manager to manage your project with ease. We will teach you how to build new tools to solve problems for you. We will teach you how to use a project manager to manage your project with ease. We will teach you how to use a project manager to manage your project with ease. We will teach you how to use a project manager to manage your project with ease.\n\n\nWe will teach you how to build your own tools to solve your own problems. We will teach you how to use a project manager to manage your project with ease. We will teach you how to use a project manager to manage your project with ease. We will teach you how to build your own tools to solve your own problems. We will teach you how to use a project manager to manage your project with ease. We will teach you how to use a project manag

In [21]:
result = generation(
    "in this course. We will teach you how to",
    pad_token_id=generation.tokenizer.eos_token_id
) 
print(result[0]['generated_text'])

in this course. We will teach you how to create virtual objects, how to use virtual controllers to implement your application, and how to create classes, functions, and properties. We will cover the basics of creating virtual objects, how to create virtual controllers, and how to use virtual services. We'll discuss the fundamentals of virtual objects and how to use services to create virtual objects. We'll also cover the various ways to use virtual services to create virtual objects and how to use virtual services. We'll discuss the concepts of virtual methods, virtual objects, and virtual virtual functions. We'll also learn how to use virtual functions to create virtual functions and how to use virtual services. We'll also learn how to use virtual functions with virtual objects. And we'll get you started on this course!

Virtual Objects

In this course, we'll learn how to create virtual objects, how to use virtual controllers, how to use virtual services, and how to use virtual servic

In [5]:
from transformers import pipeline
generation = pipeline("text-generation", "skt/kogpt2-base-v2")
result = generation(
    "이 과정은 다음과 같은 방법을 알려드려요. ",
    pad_token_id = generation.tokenizer.eos_token_id,
    max_new_tokens = 200, # 생성할 최대 길이(생성할 토큰 수)
#     num_return_sequences=1,    # 생성할 문장 갯수
#     do_sample=True,            # 다양한 샘플 사용
#     top_k=50,            # top-k 샘플링(확률 높은 상위 50개 토큰만 사용)         
#     top_p=0.95,          # 확률이 높은 순서대로 95%가 될 때까지의 단어들로만 후보로 사용
#     temperature=1.0,     # 창의성 조절(낮을 수록 보수적)
#     no_repeat_ngram_size=2 # 반복 방지
)
print(result[0]['generated_text'])

Device set to use cpu


이 과정은 다음과 같은 방법을 알려드려요. 뭘 하든 상관없으니까요."
그러나 그는 그런 말을 듣고는 말을 이었다.
"나도 그 이야기를 들었어요."
"그런데 나는 아무것도 묻지 않았어요. 단지 나보다 더 잘 알 수 있는 사람들을 봤기 때문이에요."
"그러니까 그 사람들보다는 그 사람들의 성격이 더 좋다고 생각해요."
"그래서 그 사람은 어떤 사람이었어요?"
"그렇지요. 나는 그 사람의 성격이 아주 좋았어요."
"그 사람은 아주 잘 알 수 있었어요."
"그 사람은 아주 잘 알 수 있었어요."
"그 사람은 아주 잘 알고 있었어. 하지만 그 사람은 아주 잘 알 수 있었어. 그 사람은 아주 잘 알고 있었어. 그 사람은 아주 잘 알고 있었어."
"그 사람은 아주 잘 알고 있었어. 그리고 그 사람은 아주 잘 알고 있었어."
"그 사람은 정말 잘 알고 있었어요."
"그 사람은 아주 잘 알고 있었어. 그리고 그 사람은 아주 잘 알고 있었어. 그런데 나는 그


## 4. 마스크(빈칸) 채우기

In [11]:
unmasker = pipeline(task='fill-mask',
                   model='distilbert/distilroberta-base') # 마스크 채우기
unmasker("I'm going to hospital and meet a <mask>", top_k=2) # top_k 기본값 5

[{'score': 0.19275875389575958,
  'token': 3299,
  'token_str': ' doctor',
  'sequence': "I'm going to hospital and meet a doctor"},
 {'score': 0.06794668734073639,
  'token': 27321,
  'token_str': ' psychiatrist',
  'sequence': "I'm going to hospital and meet a psychiatrist"}]

In [13]:
#unmasker("병원에 가서 <mask>를 만날 거예요")

In [14]:
unmasker("Hello, I'm a <mask> model.")

[{'score': 0.0361194983124733,
  'token': 265,
  'token_str': ' business',
  'sequence': "Hello, I'm a business model"},
 {'score': 0.02683814987540245,
  'token': 18150,
  'token_str': ' freelance',
  'sequence': "Hello, I'm a freelance model"},
 {'score': 0.024143634364008904,
  'token': 774,
  'token_str': ' role',
  'sequence': "Hello, I'm a role model"},
 {'score': 0.02224765345454216,
  'token': 2734,
  'token_str': ' fashion',
  'sequence': "Hello, I'm a fashion model"},
 {'score': 0.0169452466070652,
  'token': 6429,
  'token_str': ' lady',
  'sequence': "Hello, I'm a lady model"}]

In [18]:
unmasker("안녕하세요? 나는 <mask> 모델이예요.", top_k=3)

[{'score': 0.14130696654319763,
  'token': 35,
  'token_str': ':',
  'sequence': '안녕하세요? 나는: 모델이예요.'},
 {'score': 0.1223805844783783,
  'token': 116,
  'token_str': '?',
  'sequence': '안녕하세요? 나는? 모델이예요.'},
 {'score': 0.08188147097826004,
  'token': 328,
  'token_str': '!',
  'sequence': '안녕하세요? 나는! 모델이예요.'}]